In [1]:
import random

def hex_code_color():
    a = hex(random.randrange(0,256))
    b = hex(random.randrange(0,256))
    c = hex(random.randrange(0,256))
    a = a[2:]
    b = b[2:]
    c = c[2:]
    if len(a)<2:
        a = "0" + a
    if len(b)<2:
        b = "0" + b
    if len(c)<2:
        c = "0" + c
    z = a + b + c
    return "#" + z.upper()

In [9]:
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import DatetimeTickFormatter
import numpy as np
output_notebook()

# plotter
def SMA_plotter(code, df, x_axis_label):
    # create a new plot
    p = figure(title=code + " Simple Moving Averages", x_axis_label=x_axis_label, y_axis_label='Closing Price', width=800, x_axis_type='datetime')

    x_index = np.where(df.columns.values==x_axis_label)
    columns = np.delete(df.columns.values, x_index)
    print columns
    for item in columns:
        color = hex_code_color()
        if item == 'Close':
            p.line(df[x_axis_label], df[item], legend=item, line_color=color, line_width=3)
        else:
            p.line(df[x_axis_label], df[item], legend=item, line_color=color)
    
    p.xaxis.formatter=DatetimeTickFormatter(formats=dict(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    ))

    show(p, notebook_handle=True)

Loading BokehJS ...

In [30]:
# %load Moving_Average.py
"""
Created on Tue Feb 07 23:30:28 2017

@author: rkprajap
"""
import os, datetime, logging
import pandas as pd
import glob
import numpy as np

cur_dir = os.getcwd()
today = datetime.date.today()
FORMAT = '%(asctime)s - [%(levelname)s]: %(message)s'
log_file = 'log\\' + str(today) + '.log'
logging.basicConfig(format=FORMAT, filename=log_file, filemode='w', level=logging.DEBUG, datefmt='%d-%B-%Y %I:%M:%S %p')

data_files = glob.glob("data\\*.xlsx")
sma_list = [5, 10, 20, 50]
scored_df = pd.DataFrame(columns=['Code']) #this df contains final scoring

# calculate SMA from a given SMA list
def SMA_calc(smalist):
    sma_columns = ['SMA'+str(item) for item in smalist]
    sma_columns = ['Date','Close'] + sma_columns
    df = pd.DataFrame(columns=sma_columns)

    for f in data_files:
        code = f.replace('data\\', '').replace('.xlsx', '')
        stock_history = pd.read_excel(f, sheetname='Daily Data', header=0)
        for sma in sma_list:
            vars()['SMA'+str(sma)] = stock_history['Close'].rolling(window=sma,center=False).mean()
            stock_history['SMA'+str(sma)] = vars()['SMA'+str(sma)]    
            
#         Prioritize and rank items
        df = stock_history[sma_columns].tail(1)
        sorted_sma = np.sort(sma_list)

        scored_df
        score = 0
        for index, sma in enumerate(sorted_sma):
            if df['Close'].values[0] < df['SMA'+str(sma)].values[0]:
                score = score + (index + 1)*sma
                
        
        

#         df = stock_history[sma_columns].tail(np.max(sma_list))
        
        
#         print code, score, df['Close'].values[0] < df['SMA5'].values[0], df['Close'].values[0] < df['SMA10'].values[0], df['Close'].values[0] < df['SMA20'].values[0], df['Close'].values[0] < df['SMA50'].values[0]
#         SMA_plotter(code, df, 'Date')

SMA_calc(sma_list)    
    




ABB 0 False False False False
ACC 0 False False False False
ADANIPORTS 0 False False False False
AMBUJACEM 0 False False False False
APOLLOHOSP 25 True True False False
ASHOKLEY 0 False False False False
ASIANPAINT 0 False False False False
AUROPHARMA 60 False False True False
AXISBANK 5 True False False False
BAJAJFINSV 0 False False False False
BAJAJ_AUTO 25 True True False False
BAJFINANCE 5 True False False False
BANKBARODA 0 False False False False
BEL 0 False False False False
BHARATFORG 5 True False False False
BHARTIARTL 5 True False False False
BHEL 0 False False False False
BOSCHLTD 0 False False False False
BPCL 0 False False False False
BRITANNIA 5 True False False False
CADILAHC 0 False False False False
CASTROLIND 0 False False False False
CIPLA 0 False False False False
COALINDIA 0 False False False False
COLPAL 285 True True True True
CONCOR 0 False False False False
CUMMINSIND 0 False False False False
DABUR 285 True True True True
DIVISLAB 205 True False False True
DL